Demonstrate generation of multiple tones

In [ ]:
import sys
sys.path.append('../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt


In [ ]:
# After running demo_00_init.ipynb we decide which bitfile we'd like to use
bitfile = "mkids_2x2_kidsim_v2"
scan = Scan.Scan(bitfile)

In [ ]:
# Initialize simulation chain
simu = scan.simuChain
simu.analysis.qout(3)
simu.synthesis.qout(3)
simu.alloff()

# Get the kids chain
chain = scan.kidsChain

# Set the mixer of the analysis and synthesis chains to be the same
fMixer = 324
fMixerQ = chain.fq(fMixer)
scan.set_mixer(fMixerQ)

In [ ]:
fTone = 314
decimation = 2
g = 0.9

qout = 2
chain.analysis.qout(qout)
chain.synthesis.qout(qout)

fToneQ = chain.fq(fTone)
chain.analysis.set_decimation(decimation)
chain.analysis.source("product")
chain.set_tone(f=fToneQ, g=g, verbose=True)
[xi,xq] = chain.get_bin(fToneQ, verbose=True)

In [ ]:
plt.plot(xi)
plt.plot(xq)

In [ ]:
np.degrees(np.pi)

In [ ]:
chain.synthesis

In [ ]:
# Eventually we will call ddscfg(self, f=0, fi=0, g=0, cg=0, ch=0, comp=False)

def set_tones_delete_this(self, freqs, fis, gs, cgs, comp=False, verbose=False):
    self.qFreqs = self.fq(freqs)
    self.fis = fis
    fiDegs = np.degrees(self.fis)
    self.gs = gs
    self.cgs = cgs
    pfb_b = getattr(self.soc, self.synthesis.dict['chain']['pfb'])
    dds_b = getattr(self.soc, self.synthesis.dict['chain']['dds'])
    fmix = self.synthesis.dict['mixer']['freq']
    self.chs = pfb_b.freq2ch(self.qFreqs-fmix)
    print("self.chs =",self.chs)
    #fs = self.dict['chain']['fs'] 
    self.fOffsets = self.qFreqs - fmix - pfb_b.ch2freq(self.chs)
    dds_b.alloff()
    for fOffset,fiDeg,g,cg,ch in zip(self.fOffsets, fiDegs, gs, cgs, self.chs):
        dds_b.ddscfg(f=fOffset*1e6, fi=fiDeg, g=g, cg=cg, ch=ch, comp=comp)

freqs = np.array([fTone])
fis = np.zeros(len(freqs))
gs = 0.9*np.ones(len(freqs))/len(freqs)
cgs = None
#set_tones(chain,freqs,fis,gs,cgs)
chain.set_tones(freqs,fis,gs,cgs)

In [ ]:
[xi,xq] = chain.get_bin(fToneQ, verbose=True)

In [ ]:
plt.plot(xi)
plt.plot(xq)

In [ ]:
freqs = np.array([fTone+12.345, fTone])
fis = np.zeros(len(freqs))
gs = 0.9*np.ones(len(freqs))/len(freqs)
cgs = None
chain.set_tones(freqs,fis,gs,cgs)

In [ ]:
[xi,xq] = chain.get_bin(freqs[0], verbose=True)
plt.plot(xi[100:])
plt.plot(xq[100:])

In [ ]:
[xi,xq] = chain.get_bin(freqs[1], verbose=True)
plt.plot(xi[100:])
plt.plot(xq[100:])

In [ ]:
print(chain.qFreqs)